In [2]:
import os,sys
sys.path.append('./misc/lib/python3.7/site-packages')

import math
import time
import numpy as np
import requests
import nglview as nv
import ipywidgets as widgets
import matplotlib.pyplot as plt
import hublib.ui as ui
import hublib.tool as tool
from IPython.display import Javascript, display
import parmed as pmd
import re

class ExcStruc:
    def __init__(self):
        self.nres = 0
        self.resnums = []
        self.restypes = []
        self.ham = []
        self.dips = []
        self.cents = []
            
    def reset(self):
        self.nres = 0
        self.resnums = []
        self.restypes = []
        self.ham = []
        self.dips = []
        self.cents = []
            
            

def check_atoms(RefAtNames, QuerAtNames):
    ismatch = True
    for atnm in RefAtNames:
        if(QuerAtNames.count(atnm)!=1):
            ismatch = False
    return ismatch

def find_porph(pmdstruc, xtruc):
    PORatList = list(['NA', 'NB', 'NC', 'ND'])
    for n in range(0, len(pmdstruc.residues)):
        r = pmdstruc.residues[n]
        nmlist = list()
        for at in r:
            nmlist.append(at.name)
        if(check_atoms(PORatList, nmlist)):
            if(nmlist.count('MG')==1):
                xstruc.resnums.append(n)
                xstruc.nres += 1 
                xstruc.restypes.append('CHL')
                
            else:
                xstruc.resnums.append(n)
                xstruc.nres += 1 
                xstruc.restypes.append('PHO')

    return


def find_dipoles(struc,xtruc):
    ResNums = xtruc.resnums
    DipMat = np.zeros((len(ResNums),3))
    for n in range(0, len(ResNums)):
        r = struc.residues[ResNums[n]]
        for at in r:
            if at.name=='NB':
                NB = struc.coordinates[at.idx]
            if at.name=='ND':
                ND = struc.coordinates[at.idx]
        DipMat[n,:] = NB - ND
    xtruc.dips = DipMat

def find_centers(struc, xtruc):
    ResNums = xtruc.resnums
    CentMat = np.zeros((len(ResNums),3))
    for n in range(0, len(ResNums)):
        r = struc.residues[ResNums[n]]
        cent = 0.0
        for at in r:
            if at.name=='NB':
                cent += 0.5*struc.coordinates[at.idx]
            if at.name=='ND':
                cent += 0.5*struc.coordinates[at.idx]
        CentMat[n,:] = cent
    xtruc.cents = CentMat


def std_rep(nglview, nvstruc, xstruc):
    pdbview.add_trajectory(struc)
    pdbview.clear(0)
    chltxt = ''
    photxt = ''
    for n in range(0, len(xstruc.resnums)):
        if xstruc.restypes[n]=='CHL':
            if len(chltxt)>0:
                chltxt += ' or '
            chltxt += str(xstruc.resnums[n]+1)
        if xstruc.restypes[n]=='PHO':
            if len(photxt)>0:
                photxt += ' or '
            photxt += str(xstruc.resnums[n]+1)
            
        
    pdbview.add_representation('cartoon', selection='protein', color='lightblue', opacity=0.25)
    if len(chltxt)>0:
        pdbview.add_representation('licorice', selection='('+chltxt+ ')', color='green')
        pdbview.add_representation('spacefill', selection='('+chltxt+ ') and (.MG)', color='magenta')
    if len(photxt)>0:
        pdbview.add_representation('licorice', selection='('+photxt+ ')', color='blue')
    for n in range(0, np.shape(xstruc.dips)[0]):
        cent = xstruc.cents[n,:]
        dip = xstruc.dips[n,:]
        v1 = cent - 1.5*dip
        v2 = cent + 1.5*dip
        pdbview.shape.add_arrow(v1.tolist(), v2.tolist(), [1,0,0 ], 1.0 )
        
pdbid = widgets.Text(
    value='2DRE',
    placeholder='Type something',
    description='PDB ID:',
    layout = widgets.Layout(width='4cm'),
    disabled=False
)

pdbgo = widgets.Button(
    description='Display',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to display the pdb file',
    icon='' # (FontAwesome names without the `fa-` prefix)
)

pdboutput = widgets.HTML(
    value="",
    placeholder='',
    description='',
)

Output = widgets.Output()

def clear_stage(view):
    view._clear_component_auto_completion()
    if view._trajlist:
        for traj in view._trajlist:
            view._trajlist.remove(traj)
    for component_id in view._ngl_component_ids:
        component_index = view._ngl_component_ids.index(component_id)
        view._ngl_component_ids.remove(component_id)
        view._ngl_component_names.pop(component_index)
        view._remote_call('removeComponent',
            target='Stage',
            args=[component_index,])
    view._remote_call('removeAllComponents',
            target='Stage',
            args=[0])
    view._update_component_auto_completion()


def pdbgo_onclick(b):
    global struc
    global estruc
    url = 'http://files.rcsb.org/download/'+pdbid.value+'.pdb'
    r = requests.get(url, allow_redirects=True)
    if(r.status_code==200):
        fname = pdbid.value + '.pdb'
        wfd = open(fname, 'wb')
        wfd.write(r.content)
        wfd.close()
        pdboutput.value = ''
        
        xstruc.reset()
        clear_stage(pdbview)
        struc = pmd.load_file(fname)
        find_porph(struc, xstruc)
        find_dipoles(struc, xstruc)
        find_centers(struc, xstruc)
        std_rep(pdbview, struc, xstruc)

    else:
        pdboutput.value = 'Please enter a valide PDB ID code.'

xstruc = ExcStruc()
pdbgo.on_click(pdbgo_onclick)
pdbid.on_submit(pdbgo_onclick)
pdbview = nv.NGLWidget()
pdbview._set_size('500px', '500px')
pdbbox = widgets.HBox([pdbid, pdbgo])

display(pdbbox)
display(pdboutput)
display(pdbview)
struc = pmd.structure

<IPython.core.display.Javascript object>

HTML(value='', placeholder='')

NGLWidget()

In [36]:
N = len(xstruc.cents) #size of coupling matrix based on number of dipoles
V = np.zeros((N,N)) #coupling matrix

for m in range (0, N):
    for n in range (0, N):
        mu1 = xstruc.dips[m]
        mu2 = xstruc.dips[n]
        Rmn = xstruc.cents[m]-xstruc.cents[n]
        rmn = np.linalg.norm(Rmn)
        Rmnhat=Rmn/rmn
        V[m,n] = ((np.dot(mu1,mu2))-3*(np.dot(mu1,Rmnhat))*(np.dot(mu2,Rmnhat)))/(rmn**3)
        
V[np.isnan(V)] = 0  #I was recieving values 'nan' on diagonals    
#V=np.nan_to_num(V) #I was recieving values 'nan' on diagonals
print(V)

[[0.         0.01848166 0.00130203 0.00402986]
 [0.01848166 0.         0.00402109 0.00151936]
 [0.00130203 0.00402109 0.         0.01812681]
 [0.00402986 0.00151936 0.01812681 0.        ]]


In [1]:
#Calculate Dipole Orientation Matrix 
print(xstruc.dips)

#Calculate Dipole Location
print(xstruc.cents)

NameError: name 'xstruc' is not defined

In [ ]:
#Calculate R
R = []
for n in range(0,len(xstruc.cents)):
    R1=[]
    for m in range(0, len(xstruc.cents)):
        vec = np.subtract(xstruc.cents[m], xstruc.cents[n])
        ans = np.linalg.norm(vec)
        R1.append(ans)
    R.append(R1)
print(R)

In [ ]:
r = np.power(R, 3)
print(r)

In [ ]:
#Calculate Dot Product
Mu = []
for n in range(0,len(xstruc.cents)):
    Mu1=[]
    for m in range(0, len(xstruc.cents)):
        dot = np.sum(xstruc.dips[m]*xstruc.dips[n])
        Mu1.append(dot)
    Mu.append(Mu1)
print(Mu)

In [ ]:
#Calculate R hat
Rh = []
for n in range(0,len(xstruc.cents)):
    Rh1=[]
    for m in range(0, len(xstruc.cents)):
        vect = np.subtract(xstruc.cents[m], xstruc.cents[n])
        ???
        Rh1.append(vect)
    Rh.append(Rh1)
print(Rh)

In [ ]:
answ =[]
y = []
    y([n,m]) = Rh([n,m])/R([n,m])
answ.append(y)


In [ ]:
#Calculate R hat
R2 = []
for n in range(0,len(xstruc.cents)):
    R3=[]
    answ = []
    for m in range(0, len(xstruc.cents)):
        vect = np.subtract(xstruc.cents[m], xstruc.cents[n])
        R3.append(vect)
    R2.append(R3)
    #answ = R2[n,m]/R[n,m] #how do i make matrix divide R2(1,3) by R(1,3)?
    
print(R2)

In [ ]:
#Upload Button
def done_cb(w, name):
    print("%s uploaded" % name)
    w.reset()
    
f = ui.FileUpload("Protein Data Bank file", 
               "PDB File Upload",
               dir='tmpdir',
               cb=done_cb,
               maxsize='100M')

#Run All Below Button
def run_all(ev):
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.ncells())'))
button = widgets.Button(description="Run all below")
button.on_click(run_all)
display(f)
display(button)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Button(description='Run all below', style=ButtonStyle())

['tmpdir/2dre.pdb'] uploaded


In [ ]:
Prfile=f.list()
Prfiletext=Prfile[0]
        
#ParsePDB

#Matrix for ND atom coordinates
search=open(Prfiletext,'r').readlines()
NDMatrix=[]
for line in search:
    if re.match("(HETATM)(.*ND)", line):
        y = line.split()
        if len(y)==11:
            ChlID=str(y[3])+str(y[4])
            x_coord=float(y[5])
            y_coord=float(y[6])
            z_coord=float(y[7])
            Chl_atom=y[10]
        if len(y)==10:
            ChlID=str(y[3])
            x_coord=float(y[4])
            y_coord=float(y[5])
            z_coord=float(y[6])
            Chl_atom=y[9]
        coord=[x_coord, y_coord, z_coord]
        NDMatrix.append(coord)
        print(NDMatrix)
    
#Matrix for NB atom coordinates
search=open(Prfiletext,'r').readlines()
NBMatrix=[]
for line in search:
    if re.match("(HETATM)(.*NB)", line):
        y = line.split()
        if len(y)==11:
            ChlID=str(y[3])+str(y[4])
            x_coord=float(y[5])
            y_coord=float(y[6])
            z_coord=float(y[7])
            Chl_atom=y[10]
        if len(y)==10:
            ChlID=str(y[3])
            x_coord=float(y[4])
            y_coord=float(y[5])
            z_coord=float(y[6])
            Chl_atom=y[9]
        coord=[x_coord, y_coord, z_coord]
        NBMatrix.append(coord)
        print(NBMatrix)

In [ ]:
#Calculate Dipole
D = np.subtract(NDMatrix, NBMatrix)
print(D)

In [ ]:
R = []
for n in range(0,len(D)):
    R1=[]
    for m in range(0, len(D)):
        ans = np.sqrt(np.sum(np.square(np.subtract(D[n],D[m]))))
        R1.append(ans)
    R.append(R1)
print(R)


In [ ]:
V=np.zeros(len(D),len(D))
print(V)

In [ ]:
ND=np.transpose(NDMatrix)
z = np.dot(NBMatrix,ND)
print(z)